In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/stock_price_4_fun/notebook


In [2]:
import requests
import re
import time
import random
import urllib3

urllib3.disable_warnings()
from src.summarize_text import *


Selected device: cuda


In [3]:
speech_to_text = SpeechSummaryProcessor(audio_path='data/audio_ogg.ogg')
text = speech_to_text.generate_speech_to_text()

Selected device: cuda


In [4]:
text

"1: I'll be home later at 8 o'clock to pack my clothes\n\n2: At 9 o'clock I go to bed to write my essay and at 10 o'clock I go to bed\n"

In [5]:
symbol = 'SSI'

In [90]:


class NewsScraper:
    '''
    Scape News from https://vnexpress.net/ 
    How to run selenium on linux
    https://cloudbytes.dev/snippets/run-selenium-and-chrome-on-wsl2#:~:text=With%20Selenium%20libraries%2C%20Python%20can,using%20Python%20and%20Selenium%20webdriver.
    
    '''
    def __init__(self):
        pass

    def search_stock_news(self, symbol:str = "SSI",
                          date_format:Literal['day', 'week', 'month', 'year']='day')-> list : 
        symbol = symbol.upper()
        url = f"https://timkiem.vnexpress.net/?search_f=&q={symbol}&date_format={date_format}&"

        # Send a GET request to the webpage
        response = requests.get(url)
        # Check if the request was successful (status code 200)
        attemp = 0
        max_attemps = 3
        news_urls = []

        while attemp <= max_attemps:
            try:
                soup = BeautifulSoup(response.text, 'html.parser')
                # Assuming the news articles are wrapped in <article> elements
                articles = soup.find_all('article', class_='item-news-common')
                # Iterate through each article and extract the URL
                for article in articles:
                    url = article.get('data-url')
                    if url:
                        news_urls.append(url)

                break
                
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                print("Resetting the Scraper in 10 seconds...")
                time.sleep(10)  
                attemp += 1
                if attemp > max_attemps:
                    print("Max attempts reached. Exiting.")
                    break

        return news_urls

    def search_top_news_cafef(self)-> list:

        url = 'https://cafef.vn/'
        # Send a GET request to the webpage
        response = requests.get(url)
        # Check if the request was successful (status code 200)
        attemp = 0
        max_attemps = 3
        news_urls = []

        while attemp <= max_attemps:
            try:
                soup = BeautifulSoup(response.text, 'html.parser')

                # Find top news elements
                top_news_elements = soup.find_all('div', class_='top_noibat')
                top_news = top_news_elements[0] if top_news_elements else None

                top_news_links = top_news.find_all('a')
                # Extract and print the links
                for link in top_news_links:
                    news_link = link.get('href')
                    news_link = f'{url}{news_link}'
                    news_urls.append(news_link)

                    # Convert set to list
                    news_urls = list(set(news_urls))
                break
                
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                print("Resetting the Scraper in 10 seconds...")
                time.sleep(10)  
                attemp += 1
                if attemp > max_attemps:
                    print("Max attempts reached. Exiting.")
                    break

        return news_urls
    
    def search_top_news_vnexpress(self)-> list:

        url = 'https://vnexpress.net/kinh-doanh'
        # Send a GET request to the webpage
        response = requests.get(url)
        # Check if the request was successful (status code 200)
        attemp = 0
        max_attemps = 3
        news_urls = []

        while attemp <= max_attemps:
            try:
                # Parse the HTML content
                soup = BeautifulSoup(response.text, 'html.parser')

                # Find news elements
                news_elements = soup.find_all('h3', class_='title_news')
                news_elements.append(soup.find('section', class_='section_topstory_folder'))

                # Extract and return the URLs
                news_urls = [element.find('a').get('href') for element in news_elements if element.find('a')]
                break
                
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                print("Resetting the Scraper in 10 seconds...")
                time.sleep(10)  
                attemp += 1
                if attemp > max_attemps:
                    print("Max attempts reached. Exiting.")
                    break

        return news_urls
    
    def search_top_news(self)-> list:
        news_urls = self.search_top_news_cafef()
        news_urls.extend(self.search_top_news_vnexpress())
        return news_urls
    
    def take_text_from_link(self, news_url:str) -> str :
        news_url = [news_url]
        loader = NewsURLLoader(urls=news_url, 
                            post_processors=[clean_extra_whitespace],)
        news_text = loader.load()
        news_text = news_text[0].page_content
        return  news_text

In [91]:
news_scraper = NewsScraper()
# news_list = news_scraper.search_stock_news(symbol=symbol, date_format='year')
news_list = news_scraper.search_top_news()
news = news_scraper.take_text_from_link(news_url=news_list[0])

In [92]:
news_list

['https://cafef.vn//khoi-ngoai-quay-dau-ban-rong-nghin-ty-trong-tuan-dau-tien-nam-2024-dau-la-tam-diem-xa-hang-188240106022231686.chn',
 'https://cafef.vn//thi-phan-moi-gioi-hose-quy-4-2023-tcbs-vuot-vndirect-hsc-day-mbs-ra-khoi-top-5-188240105183615455.chn',
 'https://cafef.vn//ong-pham-nhat-vuong-lam-tgd-vinfast-188240106100255167.chn',
 'https://cafef.vn//ngay-nay-2-nam-truoc-vn-index-lap-dinh-lich-su-152857-diem-188240105233819823.chn',
 'https://cafef.vn//thi-truong-bat-dong-san-khat-nguon-cung-gia-nha-tang-lien-tuc-du-bao-kho-giam-188240106065023587.chn',
 'https://vnexpress.net/dai-gia-xang-dau-mien-tay-no-thue-hon-1-000-ty-dong-4697649.html',
 'https://vnexpress.net/ong-pham-nhat-vuong-truc-tiep-dieu-hanh-vinfast-4697659.html',
 'https://vnexpress.net/dai-gia-tai-chinh-trung-quoc-nop-don-xin-pha-san-4697608.html',
 'https://vnexpress.net/lam-phat-chau-au-tang-toc-tro-lai-sau-8-thang-4697666.html']

In [71]:
news

'Thị trường chứng khoán Việt Nam ghi nhận tuần giao dịch đầu năm 2024 tương đối khởi sắc. VN-Index thậm chí có thời điểm vượt mốc 1.160 điểm trước khi kết tuần giữ vững mốc 1.150 với thanh khoản dồi dào. Dòng tiền nhiều phiên đặc biệt tại nhóm vốn hoá lớn như ngân hàng. Kết tuần, chỉ số chính tăng 2,19% so với tuần cuối năm 2023 lên mức 1.154,68 điểm.\n\nDù vậy, giao dịch khối ngoại lại không mấy khả quan. Sau tuần cuối năm trở lại mua ròng, nhà đầu tư nước ngoài nhanh chóng quay đầu bán ròng trong tuần giao dịch đầu năm 2024. Giá trị bán ròng cả tuần 2-5/1 đạt 1.190 tỷ đồng, trong đó bán ròng khớp lệnh 705 tỷ và bán ròng thoả thuận 486 tỷ đồng.\n\nXét riêng trên từng sàn trong tuần, nhà đầu tư nước ngoài bán ròng luỹ kế 998 tỷ đồng trên HoSE, bán ròng 24 tỷ đồng trên HNX và bán ròng 168 tỷ đồng trên sàn UPCoM.\n\nThống kê trên từng mã, hai chứng chỉ quỹ FUESSVFL và FUEVFVND lọt top bị khối ngoại bán ròng mạnh nhất tuần đầu năm với giá trị lần lượt đạt 396 tỷ và 185 tỷ đồng. Bên cạnh đ

In [72]:
news

'Thị trường chứng khoán Việt Nam ghi nhận tuần giao dịch đầu năm 2024 tương đối khởi sắc. VN-Index thậm chí có thời điểm vượt mốc 1.160 điểm trước khi kết tuần giữ vững mốc 1.150 với thanh khoản dồi dào. Dòng tiền nhiều phiên đặc biệt tại nhóm vốn hoá lớn như ngân hàng. Kết tuần, chỉ số chính tăng 2,19% so với tuần cuối năm 2023 lên mức 1.154,68 điểm.\n\nDù vậy, giao dịch khối ngoại lại không mấy khả quan. Sau tuần cuối năm trở lại mua ròng, nhà đầu tư nước ngoài nhanh chóng quay đầu bán ròng trong tuần giao dịch đầu năm 2024. Giá trị bán ròng cả tuần 2-5/1 đạt 1.190 tỷ đồng, trong đó bán ròng khớp lệnh 705 tỷ và bán ròng thoả thuận 486 tỷ đồng.\n\nXét riêng trên từng sàn trong tuần, nhà đầu tư nước ngoài bán ròng luỹ kế 998 tỷ đồng trên HoSE, bán ròng 24 tỷ đồng trên HNX và bán ròng 168 tỷ đồng trên sàn UPCoM.\n\nThống kê trên từng mã, hai chứng chỉ quỹ FUESSVFL và FUEVFVND lọt top bị khối ngoại bán ròng mạnh nhất tuần đầu năm với giá trị lần lượt đạt 396 tỷ và 185 tỷ đồng. Bên cạnh đ

In [73]:
new_summarizer = NewsSummarizer()
sum_text = new_summarizer.summary_news(news= news)

Your max_length is set to 230, but your input_length is only 19. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)


In [9]:
print(sum_text)

SSI Research cho rằng việc phát hành tín phiếu điều chỉnh thanh khoản ngắn hạn không có nghĩa là Ngân hàng Nhà nước sẽ thắt chặt chính sách tiền tệ. 
Khối lượng phát hành lần đầu đạt 10.000 tỷ đồng trong 3 phiên đầu tiên. 
Trên thực tế, việc phát hành tín phiếu kho bạc cũng có thể coi là tích cực.

Tăng trưởng tín dụng tính đến ngày 15/9 chỉ ở mức 5,5% so với đầu năm. 
Ngân hàng Nhà nước chọn phương án phát hành tín phiếu kho bạc bắt đầu từ năm 2023 thay vì bán dự trữ ngoại hối như năm 2022. 
Điều này rất có thể là do hoạt động đầu cơ tỷ giá.



In [10]:
watch_list=['GEX', 'VIX', 'HHV', 'BSR', 'SHS', 'PDR', 'CTR', 'HAH', 'VNINDEX', 'VND']

In [94]:

class StockNewsDatabase:
    def __init__(self, summary_news_data_path='data/summary_stock_news.json'):
        self.summary_news_data_path = summary_news_data_path
        self.news_scraper = NewsScraper()
        self.new_summarizer = NewsSummarizer()

    def update_top_news(self):
        '''Update the top news on schedule'''
        summary_data = []

        news_list = self.news_scraper.search_top_news()[:8]
        for news_url in news_list:
            news = self.news_scraper.take_text_from_link(news_url=news_url)
            sum_text = self.new_summarizer.summary_news(news=news)

            # Append data to summary_data list
            summary_data.append({
                "type": "top_news",
                "stock": None,
                "summary_text": sum_text,
                "news_url": news_url
            })

        # Save the summary data to a JSON file
        self._save_summary_data(summary_data)

    def update_stock_news(self, watch_list):
        '''Update the news summary on schedule'''
        summary_data = []

        for stock in watch_list:
            news_list = self.news_scraper.search_stock_news(symbol=stock, date_format='month')[:1]
            for news_url in news_list:
                news = self.news_scraper.take_text_from_link(news_url=news_url)
                sum_text = self.new_summarizer.summary_news(news=news)

                # Append data to summary_data list
                summary_data.append({
                    "type": "stock_news",
                    "stock": stock,
                    "summary_text": sum_text,
                    "news_url": news_url
                })

        # Save the summary data to a JSON file
        self._save_summary_data(summary_data)

    def read_summary_stock_news(self):
        try:
            with open(self.summary_news_data_path, 'r', encoding='utf-8') as json_file:
                data = json.load(json_file)
                return data
        except FileNotFoundError:
            print(f"Error: File {self.summary_news_data_path} not found.")
            return None

    def extract_text_for_stock(self, stock_symbol):
        summary_data = self.read_summary_stock_news()
        if summary_data is None:
            return None, None
        
        for entry in summary_data:
            if entry["stock"] == stock_symbol:
                return entry["summary_text"], entry["news_url"]

        print(f"No summary text found for stock: {stock_symbol}")
        return None, None

    def get_all_stocks(self):
        summary_data = self.read_summary_stock_news()

        if summary_data:
            all_stocks = set(entry["stock"] for entry in summary_data)
            return list(all_stocks)
        else:
            return []

    def _save_summary_data(self, summary_data):
        check_path(self.summary_news_data_path)
        with open(self.summary_news_data_path, 'w', encoding='utf-8') as json_file:
            json.dump(summary_data, json_file, ensure_ascii=False, indent=2)
        print(f"Summary data saved to {self.summary_news_data_path}")

In [95]:
news_db = StockNewsDatabase()
# news_db.update_stock_news(watch_list)

In [100]:
news_db.extract_text_for_stock(stock_symbol='HAH')

('Cổ phiếu GMD của Gemadept là một trong những cổ phiếu tăng mạnh nhất sàn HoSE khi đóng cửa tăng toàn bộ lên 73.000 đồng. \nCác cổ phiếu cảng biển, logistics khác cũng giao dịch sôi động như HAH tăng gần 4%. \nChứng khoán cũng mở cửa phiên đầu tiên trong trạng thái tích cực.\n\nỞ nhóm ngân hàng, sắc xanh cũng chiếm ưu thế. \nVPB, BID, CTG tăng gần 2%, VIB, HDB, TCB, STB tăng trên 1%. \nNhóm bán lẻ và hàng tiêu dùng cũng nhận được sự quan tâm.\n',
 'https://vnexpress.net/co-phieu-van-tai-bien-noi-song-4693192.html')

In [97]:
news_db.get_all_stocks()

['VIX', 'HAH']

In [99]:
news_db.update_stock_news(watch_list)

Your max_length is set to 230, but your input_length is only 105. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=52)


Summary data saved to data/summary_stock_news.json


In [101]:
news_db.update_top_news()

/root/code_Bao/stock_price_4_fun/venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Your max_length is set to 230, but your input_length is only 19. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)
Your max_length is set to 230, but your input_length is only 66. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=33)
Your max_length is set to 230, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)


Summary data saved to data/summary_stock_news.json
